In [20]:
import numpy as np
import pandas as pd
import pickle as pk
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
input_data = pk.load(open('input_data.pkl', 'rb'))
target_data = pk.load(open('target_data.pkl', 'rb'))

layers = []
dimention = []
score = []

for lay in range(1,9):
    for dim in np.power(2, np.arange(12)):
        print(lay, dim)
        scores = []
        for i in range(5):
            log_input = []
            rand_lstm = Sequential()
            rand_lstm.add(LSTM(dim, activation='relu', return_sequences=True, input_shape=(1, input_data[0].shape[1])))
            for l in range(lay-1):
                rand_lstm.add(LSTM(dim, activation='relu', return_sequences=True))
            rand_lstm.compile(optimizer='adam', loss='mse')
            for input_batch in input_data:
                input_batch = input_batch.reshape(input_batch.shape[0], 1, input_batch.shape[1])
                batch_out = rand_lstm.predict(input_batch)
                log_input.append(batch_out[-1,:][0])
                rand_lstm.reset_states()
            print("made data")
            log_input = np.stack(log_input)
            cvlr = LogisticRegressionCV(Cs=1, cv=10, scoring='roc_auc', max_iter=10000)
            each_score = []
            for t in np.array(target_data).T:
                t = t.astype('int')
                cvlr.fit(log_input, t)
                each_score.append(np.mean(cvlr.scores_[1]))
            scores.append(np.mean(each_score))
        layers.append(lay)
        dimention.append(dim)
        score.append(np.mean(scores))

1 1
made data
made data
made data
made data
made data
1 2
made data
made data
made data
made data


In [ ]:
plt.figure()
for i in range(8):
    plt.plot(dimention[i*12:i*12+12], score[i*12:i*12+12], marker='.', label=f'{i+1} Layer LSTM')
plt.xlabel('Projected Dimensions')
plt.ylabel('ROC AUC')
plt.title('Bag of Random Embeddings Results')
plt.legend()
plt.show()